In [82]:
# Data Management
import pandas as pd
import numpy as np
from ta import add_all_ta_features
from pandas_datareader.data import DataReader
import yfinance as yf

# Statistics
from statsmodels.tsa.stattools import adfuller

# Unsupervised Machine Learning
from sklearn.decomposition import PCA

# Supervised Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Graphing
import matplotlib.pyplot as plt

In [83]:
# Remove Unwanted Warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None  # ignore column assingment tecnique warning (wants to use .iloc)

### Initial Data Extraction


In [84]:
# TIMEFRAME
start_date = "2017-01-01"
end_date = "2022-06-01"
symbol = "^VIX"



# FETCH USING YAHOO FINANCE 
# df = yf.download(symbol ,  start_date , end_date )


tickerData = yf.Ticker(symbol)
df = tickerData.history(period='1d', start=start_date, end=end_date)


# REFINE COLUMNS
df = data[["High" , "Low" ,  "Open", "Close" , "Volume"]]


In [86]:
# Add Technical Analysis Columns
df = add_all_ta_features( 
    df , open="Open", high="High", low="Low", close="Close", volume="Volume" , fillna=True
 )

print(f"Rows: {df.shape[0]}\nColumns: {df.shape[1]}")
df.head()

Rows: 131
Columns: 91


,High,Low,Open,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,23.76,22.730000,23.090000,22.900000,0,-0.0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,22.900000,24.729741,0.000000,0.000000
2023-01-04,23.27,21.940001,22.930000,22.010000,0,-0.0,0,0.0,-0.0,0.0,...,-0.310926,-0.062185,-0.248741,0.0,0.0,0.0,22.618800,-3.886460,-3.963999,-3.886460
2023-01-05,22.92,21.969999,22.200001,22.459999,0,0.0,0,0.0,-0.0,0.0,...,-0.394255,-0.128599,-0.265656,0.0,0.0,0.0,22.568343,2.044520,2.023900,-1.921400
2023-01-06,22.90,21.000000,22.690001,21.129999,0,0.0,0,0.0,-0.0,0.0,...,-0.924995,-0.287878,-0.637117,0.0,0.0,0.0,22.286165,-5.921638,-6.104212,-7.729260
2023-01-09,21.98,21.270000,21.750000,21.969999,0,0.0,0,0.0,-0.0,0.0,...,-1.035571,-0.437417,-0.598154,0.0,0.0,0.0,22.227981,3.975391,3.898406,-4.061137


## Feature Engineering